In [1]:
%cd /content/drive/MyDrive/HASOC2021-EnglishHindiMarathi/trainingSet

/content/drive/MyDrive/HASOC2021-EnglishHindiMarathi/trainingSet


In [2]:
pip install torch==1.3.1+cpu -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 111.8 MB 69 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu102 requires torch==1.9.0, but you have torch 1.3.1+cpu which is incompatible.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.3.1+cpu which is incompatible.


In [ ]:
pip install inltk

     |████████████████████████████████| 1.3 MB 29.3 MB/s 
     |████████████████████████████████| 78 kB 9.4 MB/s 
     |████████████████████████████████| 233 kB 70.1 MB/s 
     |████████████████████████████████| 1.2 MB 69.0 MB/s 
     |████████████████████████████████| 294 kB 71.1 MB/s 
     |████████████████████████████████| 142 kB 74.0 MB/s 
     |████████████████████████████████| 831.4 MB 2.5 kB/s 
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26324 sha256=37d0cff94017bbbb3b677b798b83152555dd7b9bd7178d57be76be2b0421c627
  Stored in directory: /root/.cache/pip/wheels/35/f3/15/01aa6571f0a72ee6ae7b827c1491c37a1f72d686fd22b43b0e
Successfully built typing
  Attempting uninstall: torch
    Found existing installation: torch 1.3.1+cpu
    Uninstalling torch-1.3.1+cpu:
      Successfully uninstalled torch-1.3.1+cpu
  Attempting uninstall: fastai
    Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.

In [3]:
import pandas as pd
import numpy as np
from glob import glob
import re
import json
import pickle

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout,BatchNormalization


from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import stemmer as hindi_stemmer

In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
english_stopwords = stopwords.words("english")
with open('marathi_stopwords.txt', encoding = 'utf-8') as f:
    marathi_stopwords = f.readlines()
    for i in range(len(marathi_stopwords)):
        marathi_stopwords[i] = re.sub('\n','',marathi_stopwords[i])
stopwords = english_stopwords + marathi_stopwords
english_stemmer = SnowballStemmer("english")

In [6]:
df=pd.read_excel("mr_Hasoc2021_train 2 (1).xlsx")

In [7]:
df.head()

,text_id,text,task_1
0,hasoc_mr_1,भारत 15 ऑगस्ट 1947 ला स्वतंत्र झाला आणि त्यानं...,NOT
1,hasoc_mr_2,स्वत ला हवा तसा बाइट किंवा प्रतिक्रिया घेण्यास...,NOT
2,hasoc_mr_3,5 व्या नंबरची अर्थव्यवस्था आहे भारताची जगात 20...,NOT
3,hasoc_mr_4,च्यायला म्हणजे दुबईचा फोन ही पुडीच निघाली की.,HOF
4,hasoc_mr_5,ह्याला खरंतर कधीच आत टाकला पाहिजे होता. पैसा आ...,HOF


In [8]:
tweets = df.text
y = df.task_1

In [9]:
regex_for_english_hindi_emojis="[^a-zA-Z#\U0001F300-\U0001F5FF'|'\U0001F600-\U0001F64F'|'\U0001F680-\U0001F6FF'|'\u2600-\u26FF\u2700-\u27BF\u0900-\u097F]"
def clean_tweet(tweet):
    tweet = re.sub(r"@[A-Za-z0-9]+",' ', tweet)
    tweet = re.sub(r"https?://[A-Za-z0-9./]+",' ', tweet)
    tweet = re.sub(regex_for_english_hindi_emojis,' ', tweet)
    tweet = re.sub("RT ", " ", tweet)
    tweet = re.sub("\n", " ", tweet)
    tweet = re.sub(r" +", " ", tweet)
    tokens = []
    for token in tweet.split():
        if token not in stopwords:
            token = english_stemmer.stem(token)
            tokens.append(token)
    return " ".join(tokens)

In [10]:
cleaned_tweets = [clean_tweet(tweet) for tweet in tweets]

In [11]:
cleaned_tweets

['भारत ऑगस्ट ला स्वतंत्र त्यानंतर तब्बल वर्षांनी जानेवारी साली भारताला राज्यघ',
 'स्वत ला हवा तसा बाइट किंवा प्रतिक्रिया घेण्यासाठी ॲन्करबाई किती हट्टाला पेटलीय जोरजबरदस्तीने तोंडात वाक्य घुसवतेय',
 'व्या नंबरची अर्थव्यवस्था भारताची जगात पर्यंत चंप्या वर अण',
 'च्यायला दुबईचा फोन पुडीच निघाली',
 'ह्याला खरंतर कधीच आत टाकला पाहिजे पैसा स्टारडमच्या जोरावर चाललाय सगळं एकदा ह्याच्यावर घाल',
 'भोसडीच्या तुझ्या आईचा नवरा तुझ्या आईची गांड बघ मागून गांड मारून मारू',
 'राज्यात हळद संसोधन प्रकिया निश्चित करण्यासाठी लोकसभा सदस्य हेमंत यांच्या अध्यक्षतेखाली अभ्यास समिती गठीत',
 'तुला कुत्र विचारत हो एकट्या कंगणाने तुमच्या गोट्या चोळून पार धूर काढला घंटा करावं क',
 '२०१४ साली महासत्ता होण्याची स्वप्न बघणारा आमचा युवक २०२० साली बेरोजगारी भत्ता मागू लागला एवढा अनर्थ evm वरच्या',
 'मॅट्रीकमध्ये चांगले गुण मिळाल्याने पुण्याच्या फग्र्युसन कॉलेजमध्ये प्रवेश त्यांना स्कॉलरशिपसह मिळाला पुढे पुण्या',
 'राम कदम वागण्यात बोलण्यात चुकला बीजेपी मग अजितदादा कुठे तुमच्या तोंडात मूतले',
 'गुरव बाई लई रांड लई माणसा

In [12]:
vectorizer = TfidfVectorizer(min_df = 5)
X = vectorizer.fit_transform(cleaned_tweets)
X = X.todense()

In [13]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

## **Logistic regression**

In [ ]:
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
y_pred = classifier.predict(X_val)

In [ ]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

         HOF       0.91      0.45      0.60       138
         NOT       0.75      0.97      0.85       237

    accuracy                           0.78       375
   macro avg       0.83      0.71      0.73       375
weighted avg       0.81      0.78      0.76       375



## **Naive Bayes**

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
y_pred = classifier.predict(X_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

         HOF       0.46      0.85      0.60       138
         NOT       0.83      0.42      0.56       237

    accuracy                           0.58       375
   macro avg       0.64      0.63      0.58       375
weighted avg       0.69      0.58      0.57       375



## **Stochastic Gradient Descent**

In [ ]:
from sklearn.linear_model import SGDClassifier

In [ ]:
classifier = SGDClassifier()
classifier.fit(X_train, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
y_pred = classifier.predict(X_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

         HOF       0.75      0.62      0.68       138
         NOT       0.80      0.88      0.84       237

    accuracy                           0.78       375
   macro avg       0.77      0.75      0.76       375
weighted avg       0.78      0.78      0.78       375



## **K Nearest Neighbours**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
classifier = KNeighborsClassifier()
classifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [ ]:
y_pred = classifier.predict(X_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

         HOF       0.44      0.73      0.55       138
         NOT       0.74      0.45      0.56       237

    accuracy                           0.55       375
   macro avg       0.59      0.59      0.55       375
weighted avg       0.63      0.55      0.55       375



## **Decision Trees**

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
y_pred = classifier.predict(X_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

         HOF       0.63      0.69      0.66       138
         NOT       0.81      0.77      0.79       237

    accuracy                           0.74       375
   macro avg       0.72      0.73      0.72       375
weighted avg       0.74      0.74      0.74       375



## **Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
classifier = RandomForestClassifier()
classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
y_pred = classifier.predict(X_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

         HOF       0.79      0.67      0.72       138
         NOT       0.82      0.89      0.86       237

    accuracy                           0.81       375
   macro avg       0.80      0.78      0.79       375
weighted avg       0.81      0.81      0.81       375



In [ ]:
#filename = 'rf_83_ma.sav'
#pickle.dump(classifier, open(filename, 'wb'))
 

In [ ]:
# load the model from disk
#loaded_model = pickle.load(open(filename, 'rb'))

## **Support Vector Machine**

In [ ]:
from sklearn.svm import SVC

In [ ]:
classifier = SVC()
classifier.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
y_pred = classifier.predict(X_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

         HOF       0.90      0.50      0.64       138
         NOT       0.77      0.97      0.86       237

    accuracy                           0.79       375
   macro avg       0.83      0.73      0.75       375
weighted avg       0.82      0.79      0.78       375



## **Ensembling-Voting**

In [ ]:
# Importing Libraries

from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Logistic Regression

lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_val)


# Support Vector Machine

svc = SVC()
svc.fit(X_train, y_train)
svc_pred = svc.predict(X_val)

# Naive Bayes

nb = GaussianNB()
nb.fit(X_train, y_train)
nb_pred = nb.predict(X_val)

# Stochastic Gradient Descent

sgd = SGDClassifier()
sgd.fit(X_train, y_train)
sgd_pred = sgd.predict(X_val)

# K Nearest Neighbour

knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_val)

# Decision Tree

dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
dt_pred = dt.predict(X_val)

# Random Forest

rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_val)

In [ ]:

# Voting

y_pred = []

for i in range(len(lr_pred)):
    one = 0
    zero = 0
    #predictions = [lr_pred[i], svc_pred[i], sgd_pred[i], dt_pred[i], rf_pred[i]]
    predictions = [lr_pred[i], svc_pred[i], nb_pred[i], sgd_pred[i], knn_pred[i], dt_pred[i], rf_pred[i]]
    # predictions = [lr_pred[i], nb_pred[i], sgd_pred[i], dt_pred[i], rf_pred[i]]
    for pred in predictions:
        if pred == 'HOF': one += 1
        if pred == 'NOT': zero +=1
    if one > zero: y_pred.append('HOF')
    else: y_pred.append('NOT')

y_pred = np.array(y_pred)

In [ ]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

         HOF       0.81      0.64      0.71       138
         NOT       0.81      0.91      0.86       237

    accuracy                           0.81       375
   macro avg       0.81      0.77      0.79       375
weighted avg       0.81      0.81      0.81       375



In [ ]:
filename = 'ensVote_82_ma.sav'
pickle.dump(classifier, open(filename, 'wb'))
 

## **XGBoost**

In [26]:
from xgboost import XGBClassifier

In [35]:
model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [36]:
y_pred = model.predict(X_val)

In [37]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

         HOF       0.93      0.51      0.66       138
         NOT       0.77      0.98      0.86       237

    accuracy                           0.81       375
   macro avg       0.85      0.74      0.76       375
weighted avg       0.83      0.81      0.79       375



In [27]:
#classifier = [xgb.XGBClassifier(learning_rate=0.1,n_estimators=1000,max_depth=15,min_child_weight=1,gamma=0,subsample=0.8,colsample_bytree=0.8,objective= 'binary:logistic',nthread=4,scale_pos_weight=1,seed=27)]

## **Neural Network**

In [ ]:
le = LabelEncoder() #label encoding labels for training Dense Neural Network
y_train = le.fit_transform(y_train)
y_val = le.transform(y_val)

In [ ]:
model = Sequential(
    [
        Dense(128, activation="relu"),
        Dropout(0.8),
        Dense(32, activation="relu"),
        Dropout(0.8),
        Dense(32, activation="relu"),
        Dropout(0.8),
        Dense(16, activation="relu"),
        Dropout(0.6),
        Dense(8, activation="sigmoid"),
        Dropout(0.4),
        BatchNormalization(),
        Dense(1, activation="sigmoid"),
    ]
)
model.compile('adam', loss='binary_crossentropy', metrics = ['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs = 50, batch_size = 32)

Epoch 1/50
47/47 [==============================] - 1s 3ms/step - loss: 0.8354 - accuracy: 0.5170
Epoch 2/50
47/47 [==============================] - 0s 3ms/step - loss: 0.8294 - accuracy: 0.5110
Epoch 3/50
47/47 [==============================] - 0s 2ms/step - loss: 0.7739 - accuracy: 0.5324
Epoch 4/50
47/47 [==============================] - 0s 2ms/step - loss: 0.7635 - accuracy: 0.5517
Epoch 5/50
47/47 [==============================] - 0s 3ms/step - loss: 0.7620 - accuracy: 0.5390
Epoch 6/50
47/47 [==============================] - 0s 3ms/step - loss: 0.7093 - accuracy: 0.5811
Epoch 7/50
47/47 [==============================] - 0s 2ms/step - loss: 0.7144 - accuracy: 0.5610
Epoch 8/50
47/47 [==============================] - 0s 3ms/step - loss: 0.7217 - accuracy: 0.5670
Epoch 9/50
47/47 [==============================] - 0s 3ms/step - loss: 0.6902 - accuracy: 0.5911
Epoch 10/50
47/47 [==============================] - 0s 3ms/step - loss: 0.6907 - accuracy: 0.5844
Epoch 11/50
47/47 [

In [ ]:
y_pred = model.predict(X_val)
y_pred = (y_pred > 0.5).astype('int64')
y_pred = y_pred.reshape(len(y_pred)) 

In [ ]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       138
           1       0.63      1.00      0.77       237

    accuracy                           0.63       375
   macro avg       0.32      0.50      0.39       375
weighted avg       0.40      0.63      0.49       375



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
filename = 'rf_83_ma.sav'
#load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
y_pred = loaded_model.predict(X_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

         HOF       0.82      0.67      0.74       138
         NOT       0.83      0.92      0.87       237

    accuracy                           0.83       375
   macro avg       0.83      0.79      0.81       375
weighted avg       0.83      0.83      0.82       375



## **Load test data**

In [38]:
%cd /content/drive/MyDrive/HASOC2021-EnglishHindiMarathi/testSet

/content/drive/MyDrive/HASOC2021-EnglishHindiMarathi/testSet


In [39]:
df_test=pd.read_csv("hasoc2021_mr_test-blind-2021.csv")

In [40]:
df_test.head()

,text_id,text
0,hasoc_mr_1875,माणसाची सर्वात मोठी समस्या इतिहासाचे दाखले द...
1,hasoc_mr_1876,मुद्दामहून थुकत बसला ते पण दोस्तांना बोलवून म...
2,hasoc_mr_1877,तुझे नोटिफिकेशन ऑन केलेत परंतु नाव दिसलं का शि...
3,hasoc_mr_1878,पांडुरंग हरी वासुदेव हरी तु तुझ्या घरी मी म...
4,hasoc_mr_1879,भेंचोद जेव्हा कोरोना कोरोना चालू होतं तेव्हा ...


In [41]:
test_tweets = df_test.text
tweet_ids = df_test.text_id

In [42]:
cleaned_test = [clean_tweet(tweet) for tweet in test_tweets]

In [43]:
X_test = vectorizer.transform(cleaned_test)
X_test = X_test.todense()


In [ ]:
lr_pred = lr.predict(X_test)
svc_pred = svc.predict(X_test)
nb_pred = nb.predict(X_test)
sgd_pred = sgd.predict(X_test)
knn_pred = knn.predict(X_test)
dt_pred = dt.predict(X_test)
rf_pred = rf.predict(X_test)

In [ ]:
# Voting

y_test = []

for i in range(len(lr_pred)):
    one = 0
    zero = 0
    #predictions = [lr_pred[i], svc_pred[i], sgd_pred[i], dt_pred[i], rf_pred[i]]
    predictions = [lr_pred[i], svc_pred[i], nb_pred[i], sgd_pred[i], knn_pred[i], dt_pred[i], rf_pred[i]]
    # predictions = [lr_pred[i], nb_pred[i], sgd_pred[i], dt_pred[i], rf_pred[i]]
    for pred in predictions:
        if pred == 'HOF': one += 1
        if pred == 'NOT': zero +=1
    if one > zero: y_test.append('HOF')
    else: y_test.append('NOT')

y_test = np.array(y_test)

In [44]:
y_test=model.predict(X_test)

In [45]:
y_test

array(['NOT', 'NOT', 'NOT', 'NOT', 'HOF', 'NOT', 'NOT', 'NOT', 'NOT',
       'NOT', 'HOF', 'NOT', 'NOT', 'NOT', 'HOF', 'NOT', 'NOT', 'NOT',
       'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT',
       'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT',
       'NOT', 'HOF', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT',
       'NOT', 'HOF', 'NOT', 'NOT', 'NOT', 'NOT', 'HOF', 'NOT', 'NOT',
       'HOF', 'NOT', 'NOT', 'NOT', 'NOT', 'HOF', 'HOF', 'NOT', 'HOF',
       'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'HOF', 'HOF', 'NOT',
       'NOT', 'HOF', 'NOT', 'NOT', 'NOT', 'NOT', 'HOF', 'NOT', 'NOT',
       'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT',
       'NOT', 'HOF', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT',
       'NOT', 'NOT', 'NOT', 'HOF', 'NOT', 'NOT', 'NOT', 'NOT', 'HOF',
       'NOT', 'NOT', 'HOF', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT',
       'HOF', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'HOF', 'NOT',
       'NOT', 'NOT',

In [46]:
submission = {'id':tweet_ids,'label':y_test}
submission = pd.DataFrame(submission)

In [47]:
submission.head()

,id,label
0,hasoc_mr_1875,NOT
1,hasoc_mr_1876,NOT
2,hasoc_mr_1877,NOT
3,hasoc_mr_1878,NOT
4,hasoc_mr_1879,HOF


In [48]:
submission.to_csv('marathi_xgb.csv', index = False)

In [ ]:
filename = 'ensVote_82_ma.sav'
loaded_model1 = pickle.load(open(filename, 'rb'))

In [ ]:
y_test=loaded_model1.predict(X_test)

In [ ]:
y_test

array(['NOT', 'HOF', 'NOT', 'NOT', 'HOF', 'NOT', 'NOT', 'NOT', 'NOT',
       'HOF', 'HOF', 'NOT', 'HOF', 'NOT', 'HOF', 'NOT', 'NOT', 'NOT',
       'NOT', 'HOF', 'NOT', 'HOF', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT',
       'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT',
       'NOT', 'HOF', 'HOF', 'NOT', 'NOT', 'HOF', 'NOT', 'NOT', 'NOT',
       'NOT', 'HOF', 'NOT', 'NOT', 'NOT', 'NOT', 'HOF', 'NOT', 'NOT',
       'HOF', 'NOT', 'NOT', 'HOF', 'NOT', 'HOF', 'HOF', 'NOT', 'HOF',
       'NOT', 'NOT', 'HOF', 'NOT', 'NOT', 'NOT', 'HOF', 'HOF', 'NOT',
       'NOT', 'HOF', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT',
       'HOF', 'HOF', 'NOT', 'HOF', 'NOT', 'NOT', 'NOT', 'HOF', 'NOT',
       'NOT', 'HOF', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT',
       'NOT', 'HOF', 'NOT', 'HOF', 'NOT', 'NOT', 'NOT', 'HOF', 'HOF',
       'NOT', 'HOF', 'HOF', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'HOF',
       'HOF', 'HOF', 'NOT', 'HOF', 'NOT', 'NOT', 'NOT', 'HOF', 'NOT',
       'NOT', 'NOT',

In [ ]:
submission = {'id':tweet_ids,'label':y_test}
submission = pd.DataFrame(submission)

In [ ]:
submission.head()

,id,label
0,hasoc_mr_1875,NOT
1,hasoc_mr_1876,HOF
2,hasoc_mr_1877,NOT
3,hasoc_mr_1878,NOT
4,hasoc_mr_1879,HOF


In [ ]:
submission.to_csv('marathi_en.csv', index = False)